# CCD Data Reduction: Part 1, darks & biases

In this notebook, we will go through the process of CCD data reduction for a fixed set of files.

Goals:

1. Learn how to load, examine, and visualize FITS files using astropy
2. Learn how to use python tools to estimate statistical parameters: mean (average), standard deviation
3. Practice using these python tools on both single files and data sets

# Index

 * [#1. Inspect Biases](#1.-Inspect-an-image:-Bias) - mostly guided work, you will run cells & read
 * [#2. Inspect Darks](#2.-Inspect-an-image:-Dark-Frame) - student work: you fill in the blanks
   * [Dark Frame Questions](#Dark-Frame-Questions)
   * [Dark Current Rate](#Dark-Current-Rate)
   * [vmin, vmax](#Student-Question:-vmin,-vmax)
   * [Bias-subtracted dark statistics](#Bias-subtracted-Dark-Stats)
   * [Noise estimate from dark stack](#Noise-estimate-from-dark-stack)
 * [#3. Image Inspection](#3.-Inspect-some-on-sky-data)

The first step is to get the data in the right place.  Running the following cell will show you where you are.  `os` stands for "operating system", and `getcwd` is short for "get current working directory"

In [ ]:
import os
print(os.getcwd())

Download your FITS files from the Canvas page and move them to the directory above.  You can do this using either the filesystem tools for your OS (e.g., Finder on mac, Windows Explorer on windows), or you can do it directly here.  For example, you might do:

```
%%bash
mv ~/Downloads/*.FIT . 
```

to move all files from your `~/Downloads` directory to the current directory (`.` and `cwd` are shorthand for the current working directory).  

Note that `~` is shorthand for your user directory, e.g., `/Users/adam` on Mac or `C:\Users\adam` on Windows.  

`*` is a wildcard that matches anything.

Verify that the expected files are all here using `ls *.FIT`.  You should see 11 bias files, `bias_00[1-9].FIT` & `bias_010.FIT`, and 30 dark files, `dark_10s_0[0-3][0-9].FIT`:

In [ ]:
ls *.FIT

# 1. Inspect an image: Bias

The first step is to look at an image and see what it contains.  We do this using [astropy's FITS reader](https://docs.astropy.org/en/stable/io/fits/) and matplotlib ([example](http://learn.astropy.org/rst-tutorials/FITS-images.html)).

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['image.origin'] = 'lower' # this is needed to have 0,0 in the lower left
plt.rcParams['image.interpolation'] = 'none' # this makes pixels less blurry
# OPTIONAL: if you want to use a different stylesheet, use this line:
#plt.style.use('dark_background')
from astropy.io import fits

In [ ]:
bias_fn = 'bias_001.FIT'
data = fits.getdata(bias_fn)

In [ ]:
plt.imshow(data)
plt.colorbar()
_=plt.xlabel("Pixel X Coordinate")
_=plt.ylabel("Pixel Y Coordinate")

Simply showing the image isn't enough, we also need to scale it so that there is reasonable contrast within the image, and we'd like it to be big enough that we can see each pixel clearly.

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(data)
plt.colorbar()
_=plt.xlabel("Pixel X Coordinate")
_=plt.ylabel("Pixel Y Coordinate")

## Show an image with improved contrast

The astropy visualization toolkit helps us improve the contrast by using a different data stretch.

In [ ]:
from astropy import visualization

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(data, norm=visualization.simple_norm(data, stretch='asinh'))
plt.colorbar()
_=plt.xlabel("Pixel X Coordinate")
_=plt.ylabel("Pixel Y Coordinate")

There are two notable features in this image:
1. There is a gradient from left to right, with the left being brighter.
2. There is a hot pixel.  How can we identify it?

Since it's the brightest pixel in the image, we can simply find the maximum value, and then find out where that maximum value is:

In [ ]:
print("Brightest value:", data.max())
print("Location of brightest value:", data.argmax())

You'll note that the pixel number above is in the 100,000's, but our image is only 765x510 (numpy arrays use reverse-ordered indexing, so it is (y,x) instead of (x,y)):

In [ ]:
data.shape

To find the location in x,y coordinates, we use a numpy tool:

In [ ]:
import numpy as np

In [ ]:
print("y,x coordinates of the brightest pixel:", np.unravel_index(data.argmax(), data.shape))

Describe in your own words: what did `unravel_index` do?

-student answer-

We can then "zoom in" to that part of the image by "slicing" out a chunk of the `data` array:

In [ ]:
plt.figure(figsize=(14,10))
# we pick a "slice" from (y-8) to (y+8) and from (x-8) to (x+8)
plt.imshow(data[300:316, 620:636], cmap='inferno')
plt.colorbar()
_=plt.xlabel("Pixel X Coordinate (+620)")
_=plt.ylabel("Pixel Y Coordinate (+300)")

We need to know something about the _distribution_ of pixel brightnesses beyond just where they are.  We can look at this distribution using a histogram.

In [ ]:
# the "ravel" method turns the 2D data image into a 1D list of values.
# we use it here because of the syntax requirements of the `hist` function
histresult = plt.hist(data.ravel(), bins=50)
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

Note that the above histogram shows a broad range along the X-axis, but most of the pixels are in a narrow range.  The hot pixel is an _outlier_ from this distribution.

We can see this more clearly by log-scaling the histogram

In [ ]:
histresult = plt.hist(data.ravel(), bins=50, log=True)
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

We can re-make this histogram excluding the outlier using a "masking" approach.

In [ ]:
# first, we create a mask that excludes the bad data
# this is a "boolean array" of True and False values
# it will be True everywhere except on the one hot pixel, where it will be False
mask = data < 1800
# then, we re-histogram selecting the "good" data
histresult = plt.hist(data[mask].ravel(), bins=50)
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

Our bias image has a gradient in it, as you could see above, so its noise distribution does not look like a Gaussian.   Instead, it has a tail.  We can see this by over-plotting a Gaussian function with a reasonable estimate for the location parameters.

The Gaussian function is:

$$
f(x) = \frac{1}{\sigma \sqrt{2 \pi }} e^{-(x-x_0)^2 / (2 \sigma^2)}
$$

In [ ]:
# this part repeats the histogram-making above
mask = data < 1800
histresult = plt.hist(data[mask].ravel(), bins=50)

# 'linspace' makes an array of (175) evenly-spaced points from the lower (1000) to upper (1175) value
x = np.linspace(1000, 1175, 175)
sigma = 15 # I guessed this value by eye
x0 = 1075 # I guessed this too
# compute the Gaussian function:
y = 1/(sigma*(2*np.pi)**0.5) * np.exp(-(x-x0)**2/(2*sigma**2))
scale = 900000 # we need to multiply by a scale factor to see the curve; this number is arbitrary
plt.plot(x, y*scale)
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

You can see from the above curve that there is something like a "skewed Gaussian" distribution in the data: the Gaussian curve fits most of the data, but there's a tail out to the right side.

Because of this tail, our noise estimates on the data won't be exactly right, but we can make them anyway.

We estimate the noise by taking the _standard deviation_ of the data.  If the bias were uniform, the standard deviation of our image would represent a measurement of the _read noise_ plus the intrinsic pixel-to-pixel variation.  If the pixel-to-pixel variation is small, the read noise would then be the only contributor to the measurement.

(`ddof` is the degrees of freedom.  This is a statistical correction for the fact that we're estimating the mean and the variation around the mean from the same data)

In [ ]:
print("Estimated read noise + pixel-to-pixel variation (without accounting for gradient):", data[mask].std(ddof=1))

The `std` method gives us the standard deviation of the data, and it suggests that the noise is about 19 counts.

However, now we need to explore another way to estimate the readnoise.

We have many bias images.  We can combine them to obtain a direct estimate of the readnoise, independent of the pixel-to-pixel variation.

Each bias image is the same.  The only difference between the images will be read noise.  That means we can estimate the noise on a pixel-by-pixel basis, but by comparing each pixel to itself over many measurements.

We start by verifying that our bias images are all taken under the same conditions.  We can do this by looking at their FITS headers to make sure they have the same exposure time.

In [ ]:
# the "glob" module lets us store the results of wildcard searches into python variables
import glob
bias_file_list = glob.glob("bias*FIT")
print("There are",len(bias_file_list),"bias images")
print(bias_file_list)

We use `for` loops to loop over the entries in a list.  This example will print relevant information - the exposure time and CCD temperature - for each "bias" image.

In [ ]:
for filename in bias_file_list:
    header = fits.getheader(filename)
    print(filename, header['EXPTIME'], header['CCD-TEMP'])

By inspecting the headers above, we see that we have 10 bias images to work with.

Note that these "bias" images are not strictly bias images; they are technically 0.04 second Dark frames.  In 0.04 seconds, we probably will not accrue much dark current, but there might be some small dark current contribution.

(aside: how do we inspect headers?  How did I know to use the `'EXPTIME'` keyword?  `fits.getheader` loads a header from a file.  In the cell below, load a header from any file and see what it looks like:)

In [ ]:
# STUDENT ACTIVITY:
# Use `fits.getheader` to read the header from a .FIT file and show the header contents
header = fits.getheader(...)
header

Because these came from the same CCD, they all have the same dimensions.

We can therefore stack up their data into a "data cube" that will effectively be a time series: two dimensions are pixel dimensions (x, y), while the third (z) is time; we took each image at a different time, and that's the only thing that changed.

In [ ]:
# the `fits.getdata` function loads data from a fits file
# np.array() converts an input to an array
# the "inline for loop" is the really important part here, though:
# we are loading one image for each file name in the bias_file_list we made above
bias_timeseries = np.array([fits.getdata(fn) for fn in bias_file_list])

In [ ]:
print(bias_timeseries.shape)

The data structure `bias_timeseries` is now an array of ten images that are each the same size.

We can perform statistics along the time axis (the array is in `z,y,x` shape, so `z` is the time axis).  To see what this means, we can look at what the time series data looks like for a single pixel.

In the example below, we plot the time series for pixel x=150, y=100:

In [ ]:
_=plt.plot(bias_timeseries[:,100,150], marker='o')
_=plt.xlabel("Frame Number")
_=plt.ylabel("Pixel Value")

We can take statistics of this pixel to see what the mean and standard deviation are:

In [ ]:
print("Mean =",bias_timeseries[:,100,150].mean(), " Standard deviation =",bias_timeseries[:,100,150].std(ddof=1))

So we see that our bias is about 1092.5 +/- 12.2 counts.

To repeat this measurement for _every_ pixel, we obtain the statistics along an axis:

In [ ]:
bias_mean = bias_timeseries.mean(axis=0)
bias_std = bias_timeseries.std(axis=0, ddof=1)

Take a moment to appreciate the above line.  It just did a _lot_ of things all at once:

* it calculated the mean of 10 exposures for each of 765 * 510 = 390150 pixels
* it calculated the standard deviation of 10 exposures for each pixel
* it stored the results of each into an image array of the same shape as the inputs

This array arithmetic - doing the same operations on millions of pixels all at once - is at the heart of image analysis.  Computers can do this sort of thing very fast, and they have to.

We then inspect these as images, as before.  The mean image looks just like the original data image:

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(bias_mean, norm=visualization.simple_norm(bias_mean, stretch='asinh'))
plt.colorbar()
_=plt.title("Mean of Bias Images")
_=plt.xlabel("Pixel X Coordinate")
_=plt.ylabel("Pixel Y Coordinate")

However, the standard deviation image looks quite different: it's now nearly the same everywhere, the only thing left in the image is (read) noise:

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(bias_std, norm=visualization.simple_norm(bias_std, stretch='asinh'))
plt.colorbar()
_=plt.title("Standard Deviation of Bias Images")
_=plt.xlabel("Pixel X Coordinate")
_=plt.ylabel("Pixel Y Coordinate")

Each value in the `bias_std` map is an independent measurement of the read noise from the detector.  Let's look at some values, including our x,y=150,100 and at the hot pixel:

In [ ]:
print("Estimate of read noise at 150,100:",bias_std[100,150],"and at y,x=(308, 628)", bias_std[308,628])

To get the best estimate of the average read noise, we can take the average *over all pixels* of our standard deviation *measured across time samples*:

In [ ]:
mean_readnoise = bias_std.mean()
print("The mean readnoise over all pixels =",mean_readnoise)

Similarly, we can compute the average bias (again, averaging over all pixels in the CCD):

In [ ]:
mean_bias = bias_mean.mean()
print("mean bias =",mean_bias)

So we have obtained an average bias of 1080 and an average read noise of 10.6.

However, the distribution function shown above was a _skewed_ Gaussian, which means our estimate of the mean and standard deviation are biased.  To account for such issues, we can employ _robust statistics_ to obtain better estimates.  "Robust statistics" are statitical estimates that are robust against outliers, i.e., adding in a few outliers does not substantially affect the estimate.

One example robust estimator is the _median_, which is very good at approximating the _mean_:

In [ ]:
median_bias = np.median(bias_mean)
print("Median bias level across the image =",median_bias)
median_readnoise = np.median(bias_std)
print("Median standard deviation level across the image =",median_readnoise)

We can look at the histogram of the read noise estimates (`bias_std`).  In this case, they are very similar:

In [ ]:
histresult = plt.hist(bias_std.ravel(), bins=50, log=True)
# plot a vertical line at the median, then at the mean
plt.vlines(median_readnoise, 1, 1e5, linestyle='--')
plt.vlines(mean_readnoise, 1, 1e5, linestyle=':')
plt.title("Read noise estimates (per-pixel bias standard deviation)")
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

For the bias value, the mean and median are quite different: the pixels are not "unimodel", they have a broad, non-Gaussian distribution.

In [ ]:
# we will re-make the mask we used to exclude outliers
mask = bias_mean < 1800
histresult = plt.hist(bias_mean[mask], bins=50, log=True)
# plot a vertical line at the median, then at the mean
plt.vlines(median_bias, 1, 1e5, linestyle='--')
plt.vlines(mean_bias, 1, 1e5, linestyle=':')
plt.title("Mean bias values")
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

# 2. Inspect an image: Dark Frame

[Index](#Index)

Next, we will look at our dark frames and see how they differ from the bias frames.

In [ ]:
# student entry here: first, load the data
# Start with the first 10s dark image
dark_fn = 'dark_10s_001.FIT'
dark_data = fits.getdata(dark_fn)

#dark_fn = '<your answer here>' # put a filename in the quotes
#dark_data = <your answer here># then, load a FITS file's data as you've done above for the bias

# note:your answers should be something like:
# dark_fn = 'some_fits_fits.fits'
# dark_data = some_function_to_get_fits_data(dark_fn)
# in other words, use the variable dark_fn

In the next cell, show the image as we did [above](#Show-an-image-with-improved-contrast) for the bias.

In [ ]:
# student entry here
# then, display the data, just as with the bias above
plt.figure(figsize=(14,10))
plt.imshow(dark_data, norm=visualization.simple_norm(dark_data, stretch='asinh'))
plt.colorbar()

## Dark Frame Questions
[Index](#Index)

Use the code examples you saw above to answer these questions in the cells below

1. Is there a hot pixel?  Where is it?  
2. How long was the exposure?  Is the filename correct or misleading?
3. What is the average value in our dark frame?
4. What is the average dark current rate?  Note that you have to subtract the average bias to determine this.  You should use the _average_ dark current and _average_ bias for this.

In [ ]:
# student exercise: find the hot pixel (again, as done above for the bias image)
- student work here - 

In [ ]:
# student exercise: find the exposure time from the FITS header
# reminder: delete the <s and >s
print("The exposure time of dark_10s_001.FIT is", <your answer here>)

In [ ]:
# student exercise: take the mean and median of the dark frame you loaded a minute ago
# (from above, the dark frame's data should be in the variable `dark_data`)
print("Mean value =", <your answer here>)
print("Median value =", <your answer here>)

You should get values close to or exactly:

 * Mean value = 1095.2488222478535
 * Median value = 1092.0

## Dark Current Rate
[Index](#Index)

In the next cell, you'll estimate the _rate_ of dark current accumulation.

A rate is simply a number per time, so you need to calculate the number of dark counts and the exposure time.

Dividing them will get you the dark rate.

In [ ]:
# student exercise: Determine the dark current and dark current rate
# the goal is to use the values computed above (the mean dark current and the mean bias)
# to obtain an estimate of the mean dark current and dark current rate
<probably write a little code here>
print("Dark current counts =", <your answer here>)
print("Dark rate (dark current / exposure time) =", <your answer here>, "counts per second")

You should get values close to or exactly:

 * Average dark current counts = 15.132245033961453
 * Average dark rate = 1.5132245033961453 counts per second

In the above exercise, you estimated the average dark current from the data.  However, we can do better by subtracting the bias pixel-by-pixel first.

Let's start by seeing what the bias-subtracted dark looks like.  As long as the images are the same `shape` (which is always true if they come from the same CCD), we can subtract them from one another.

The next several cells make images and guide you through some detailed image analysis.

In [ ]:
bias_subtracted_dark = dark_data - bias_mean

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(bias_subtracted_dark, norm=visualization.simple_norm(bias_subtracted_dark, stretch='asinh'))
plt.colorbar()
_=plt.xlabel("Pixel X Coordinate")
_=plt.ylabel("Pixel Y Coordinate")

In the image above, we still have a pesky very bright outlier that is making our display look bad. 


To fix this, we need to know something about the _distribution_ of our data again

In [ ]:
histresult = plt.hist(bias_subtracted_dark.ravel(), bins=50, log=True)
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

From the above plot, we can see that _almost_ all of the data are in the range [0,2000], but there's that one big outlier.  In fact, the vast majority of the data are below, say, 100.  Let's mask out the bad pixel and look closer.

In [ ]:
mask = bias_subtracted_dark < 8000
histresult = plt.hist(bias_subtracted_dark[mask].ravel(), bins=50, log=True)
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

This new histogram, excluding our dramatic outlier, shows us that almost all of the data are below 200 counts.  We have only about 10-20 outliers above 200 (recall that $10^0 = 1$).  So let's zoom in once more:

In [ ]:
mask = bias_subtracted_dark < 200
histresult = plt.hist(bias_subtracted_dark[mask].ravel(), bins=50, log=True)
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

The above plot is a well-behaved, Gaussian-looking distribution with a small tail, which we can see if we plot it with a linear Y-axis again:

In [ ]:
mask = bias_subtracted_dark < 200
histresult = plt.hist(bias_subtracted_dark[mask].ravel(), bins=50)
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

We can now display the image with a more reasonable range of values.  I picked -30 to 50, as they're pretty close to the boundaries of where we expect to see any pixels: there are only a couple pixels above and below these values.

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(bias_subtracted_dark, vmin=-30, vmax=50)
plt.colorbar()
_=plt.xlabel("Pixel X Coordinate")
_=plt.ylabel("Pixel Y Coordinate")

### Student Question: vmin, vmax
[Index](#Index)

What do the `imshow` arguments `vmin` and `vmax` do?

`<your answer here>`

Now we can see that the above image is a clean-looking noise frame.  There are a few outliers, but the mean value is constant across the image - there is no gradient.

Since we saw a gradient in the bias image, but we don't see one in the dark minus bias image, we can infer that the gradient is in the bias - in other words, it's intrinsic to our CCD and will always be there no matter how short an exposure we take.

## Bias-subtracted Dark Stats
[Index](#Index)

Now we can compute some statistics about this bias-subtracted dark image.  

What are the mean and median?

In [ ]:
# student exercise: compute the mean and median of the bias-subtracted-dark
print("Mean bias_subtracted_dark =", <your answer here>)
print("Median bias_subtracted_dark =", <your answer here>)

You should get:

* Mean bias_subtracted_dark = 15.132245033961297
* Median bias_subtracted_dark = 15.0

These values are an estimate of the dark current obtained in this exposure, but they're a little different from the estimate obtained taking the mean of the dark minus the mean of the bias.

What is the estimated dark current rate (counts per second) for these data?

In [ ]:
# student exercise: What is the dark current rate estimated after bias subtraction?

print("Rate estimate =", <your answer here>)

We can improve on this rate estimate by averaging over several dark images, like we did above with bias images.

In [ ]:
dark_file_list = glob.glob("dark_10s*FIT")
print("There are",len(dark_file_list),"dark images")
print(dark_file_list)

In [ ]:
for filename in dark_file_list:
    header = fits.getheader(filename)
    print(filename, header['EXPTIME'], header['CCD-TEMP'])

We have 29 dark images each with 10 second exposure times.

We then load them into a time series like we did before with the bias images:

In [ ]:
dark_timeseries = np.array([fits.getdata(fn) for fn in dark_file_list])
dark_timeseries.shape

We can again plot the dark values for some pixel:

In [ ]:
plt.plot(dark_timeseries[:,100,150], 'o')
_=plt.xlabel("Frame ID Number")
_=plt.ylabel("Pixel Value")

We calculate a noise image by taking the standard deviation in time:

In [ ]:
dark_std = dark_timeseries.std(axis=0, ddof=1)
dark_mean = dark_timeseries.mean(axis=0)

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(dark_std, norm=visualization.simple_norm(dark_std, stretch='asinh'))
plt.colorbar()
_=plt.xlabel("Pixel X Coordinate")
_=plt.ylabel("Pixel Y Coordinate")

Our hot pixel is causing problems, so let's chop it out semi-automatically using a `percentile` cut.  By setting `max_percent=99.95` below, we set the brightest pixel in the displayed stretch to be just a little bit fainter than the brightest pixel.

In [ ]:
plt.figure(figsize=(14,10))
plt.imshow(dark_std, norm=visualization.simple_norm(dark_std, stretch='asinh', max_percent=99.95))
plt.colorbar()
_=plt.xlabel("Pixel X Coordinate")
_=plt.ylabel("Pixel Y Coordinate")

### Noise estimate from dark stack
[Index](#Index)


So again we have an image that contains (almost) only noise, but this time it contains contributions from both read noise and from dark current.


What are the mean and median noise level in this image?

In [ ]:
# student exercise
mean_dark_noise = <your answer here>
median_dark_noise = <your answer here>

In [ ]:
print("Mean dark noise:",mean_dark_noise)
print("Median dark noise:",median_dark_noise)

You should get values:

 * Mean dark noise: 10.870688076255952
 * Median dark noise: 10.813465423710756


We had 29 dark frames and 10 bias frames, so our estimate of the dark noise (from the dark frames) will be more precise than the readnoise estimate from the bias frames.  In the histogram below, the distribution will be narrower for the darks.

In [ ]:
# comparison of the dark noise (note that this time we hand-specified the bin locations)
_ = plt.hist(dark_std.ravel(), bins=np.linspace(0,20), label='Dark')
_ = plt.hist(bias_std.ravel(), bins=np.linspace(0,20), label='Bias', alpha=0.5)
leg = plt.legend(loc='best')
_=plt.xlabel("Pixel Value")
_=plt.ylabel("Number of Pixels")

In [ ]:
print("Mean read noise:",mean_readnoise)
print("Median read noise:",median_readnoise)

Now we can estimate how much _noise_ was added by the dark current.

We know that the dark current added (dark_mean - bias_mean) counts:

In [ ]:
mean_dark_current = np.mean(dark_mean) - np.mean(bias_mean)
mean_dark_current

What is the resulting dark rate?

In [ ]:
# student exercise
dark_rate_estimate = <your answer here>
dark_rate_estimate

You should obtain ~1.39 counts/second

Given the dark rate you've estimated, let's assess whether the "bias" frames are really dark-electron-free.

How many dark current counts do you expect, on average, per pixel in the bias frames?

In [ ]:
# student exercise
n_dark_counts = <your answer here>
n_dark_counts

Based on that result, assess: Is the short "bias" exposure really a good estimate of the bias, or is it significantly affected by dark current?

Student answer:
-student answer-

# 3. Inspect some on-sky data

[Index](#Index)

There is one image file, `target_image.FIT`, that we'll inspect now.

First, display the image with reasonable contrast using what you've learned above.  

Recall that you can see help on a command by putting `<commandname>?` in a cell.  For example, you might do `visualization.simple_norm?` to see what the `simple_norm` function can do, or `plt.imshow?` to see what imshow can do.

In [ ]:
# student exercise: load the file
target_image = <your answer here>

In [ ]:
# student exercise: display the data so you can see what's going on
<your answer here>

You should note that there are still artifacts similar to those we saw earlier.  So, now we will subtract the dark frame from the image.  You have an average dark frame built from the average of many 10s darks.  Use that!

In [ ]:
# student exercise: make a dark-subtracted version of the above
darksub_target_image =  <your answer here>

In [ ]:
# student exercise: display the dark subtracted data
<your answer here>


What do you think?  Did dark subtraction improve the image?  How?

<student answer>

# Index (end)

 * [#1. Inspect Biases](#1.-Inspect-an-image:-Bias) - guided work, you will run cells & read
 * [#2. Inspect Darks](#2.-Inspect-an-image:-Dark-Frame) - student work: you fill in the blanks
   * [Dark Frame Questions](#Dark-Frame-Questions)
   * [Dark Current Rate](#Dark-Current-Rate)
   * [vmin, vmax](#Student-Question:-vmin,-vmax)
   * [Bias-subtracted dark statistics](#Bias-subtracted-Dark-Stats)
   * [Noise estimate from dark stack](#Noise-estimate-from-dark-stack)
 * [#3. Image Inspection](#3.-Inspect-some-on-sky-data)